In [3]:
import csv
from pandas import *
import numpy as np
import pandas as pd

# Running RoBERTA using the SiEBERT:Sentiment in English RoBERTa pipeline

In [ ]:
from transformers import pipeline

# Get siebert model and tokenizer packaged in pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [27]:
# Read in Transcriptions
f = open('./transcriptions.csv', 'r')
file = csv.DictReader(f)
sep = '_'
ids = []
sentences = []
for col in file:
    ids.append(col['ID'].split(sep,1)[0])
    sentences.append(col['Transcription'])


In [11]:
# Extract seniment of each transcription, transcriptions are truncated to 512 tokens
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
sentiment_prob = [sentiment_analysis(x,**tokenizer_kwargs) for x in sentences]

In [12]:
# Store final sentiment and corresponding probability
sentiment = []
probability = []
for sent in sentiment_prob:
    sentiment.append(sent[0]['label'])
    probability.append(sent[0]['score'])



['POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE']
[0.9988191723823547, 0.9944068193435669

In [28]:
# Create dataframe for sentiment and probabilities
df_RoBERTa_sentiment = pd.DataFrame({'VideoID': ids, 'Transcription': sentences,'Sentiment_RoBERTa': sentiment,'Probability':probability})
print(df_RoBERTa_sentiment)

                VideoID                                      Transcription   
0   7124357659771505966   I see why so many celebrities love chotted tu...  \
1   7219073048287153413   I want this much shit All the girls gonna be ...   
2   7141347073915374853     Charlotte Tilbury Magic Cream. Hydration Test.   
3   7193418545542499630   Hold on friends because I think they're tryin...   
4   7131089775624211755   If your base looks like this, you can now do ...   
..                  ...                                                ...   
75  7219098807437397291   Okay, just coming on here to give a special s...   
76  7220512664575986990              I love it. I love it more. I love it!   
77  7207173046430354734   Sunday, the most delicious day of the week. B...   
78  7227193522351246598   So my skin still has a recover from winter so...   
79  7213312295844236550   I hate that I want to recommend this because ...   

   Sentiment_RoBERTa  Probability  
0           POSITIVE     0.

In [29]:
# Read in label file
label_df = pd.read_csv('labels.csv', delimiter=';')
print(label_df)


                VideoID Sponsored Sentiment
0   7215716180294683946       Yes   Postive
1   7210487841732840746       Yes  Positive
2   7218032675850849582       Yes  Positive
3   7102205817524555054       Yes  Positive
4   7220512664575986990       Yes  Positive
..                  ...       ...       ...
75  6990818542262226181        No  Positive
76  7166197472560254213        No  Positive
77  7203417189737565446        No  Positive
78  7098279023851097349        No   Neutral
79  7182239739980958982        No   Neutral

[80 rows x 3 columns]


In [35]:
# Merge two dataframes 
df_RoBERTa_sentiment['VideoID'] = df_RoBERTa_sentiment['VideoID'].astype(int)
df_final = pd.merge(label_df, df_RoBERTa_sentiment, on="VideoID")
print(df_final)

                VideoID Sponsored Sentiment   
0   7215716180294683946       Yes   Postive  \
1   7210487841732840746       Yes  Positive   
2   7218032675850849582       Yes  Positive   
3   7102205817524555054       Yes  Positive   
4   7220512664575986990       Yes  Positive   
..                  ...       ...       ...   
75  6990818542262226181        No  Positive   
76  7166197472560254213        No  Positive   
77  7203417189737565446        No  Positive   
78  7098279023851097349        No   Neutral   
79  7182239739980958982        No   Neutral   

                                        Transcription Sentiment_RoBERTa   
0    The confidence boost, a glowy and dewy base g...          POSITIVE  \
1    Watch how I create this glowing skin makeup b...          POSITIVE   
2    I get why this cream by Charlotte Tilbury is ...          POSITIVE   
3                           Thanks for watching guys!          POSITIVE   
4               I love it. I love it more. I love it!       

In [37]:
# Write final dataframe to results.csv
df_final.to_csv('./results.csv', encoding='utf-8', index=False)